# Workload rtapp + Idle States Residency Analysis

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup(level=100)

# Execute this cell to enabled devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
%matplotlib inline

import os

# Support to access the remote target
from env import TestEnv

# Support to access cpuidle information from the target
from devlib import *

# Support to configure and run RTApp based workloads
from wlgen import RTA, Ramp, Periodic

# Support for trace events analysis
from trace import Trace
from trace import ResidencyTime, ResidencyData

# DataFrame support
import pandas as pd
from pandas import DataFrame

# Trappy (plots) support
from trappy import ILinePlot
from trappy.stats.grammar import Parser

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# CPUIdle break residency statistics

In [3]:
def dfCPUIdleStateEstimationStatistics(ia, cpu, residency):
    
    #residency = residency / 1000000
    
    idle_df = ia._dfg_trace_event('cpu_idle')
    cpu_idle = idle_df[idle_df.cpu_id == cpu]

    # Generate valid idle states
    available_idles = sorted(idle_df.state.unique())
    available_idles = available_idles[1:]

    cpu_is_idle = ia._trace.getCPUActiveSignal(cpu) ^ 1
    cpu_idle = cpu_idle.join(cpu_is_idle.to_frame(name='is_idle'),
                                 how='outer')
    cpu_idle.fillna(method='ffill', inplace=True)

    idle_time = pd.DataFrame({'over_est' : 0, 'under_est' : 0, 'good_est' : 0, 'cpu' : 0}, index=available_idles)

    for i in available_idles:
        idle_state = cpu_idle.state.apply(
            lambda x: 1 if x == i else 0
        )
        #print idle_state
        idle_t = cpu_idle.is_idle * idle_state
        #print idle_t
        # Compute total time by integrating the square wave
        #print ia._trace.integrate_square_wave(idle_t)
        idle_t.iloc[-1] = 0.0

        # Compact signal to obtain only 1-0-1-0 sequences
        comp_sig = idle_t.loc[idle_t.shift() != idle_t]
        #print comp_sig
        if comp_sig.iloc[0] == 0.0:
            idle_interval = (comp_sig.iloc[2::2].index - comp_sig.iloc[1:-1:2].index)
        else:
            idle_interval = (comp_sig.iloc[1::2].index - comp_sig.iloc[:-1:2].index)
        
        idle_interval *= 1000000
        
        idle_time.at[i, 'cpu']  = cpu
        idle_time.at[i, 'over_est']  = len(idle_interval[idle_interval < residency[i]])
        idle_time.at[i, 'under_est'] = len(idle_interval[idle_interval > residency[i+1]])
        idle_time.at[i, 'good_est']  = len(idle_interval[(idle_interval >= residency[i]) & (idle_interval < residency[i+1])])          
        
    return idle_time

In [4]:
from collections import namedtuple

ResidencyTimes = namedtuple('ResidencyTimes', ['label', 'stats'])

def dfAllCPUIdleStateEstimationStatistics(ia, residency, cpus=None):

    if not ia._trace.hasEvents('cpu_idle'):
        ia._log.warning('Events [cpu_idle] not found, '
                        'plot DISABLED!')
        return

    if cpus is None:
        # Generate plots only for available CPUs
        cpuidle_data = ia._dfg_trace_event('cpu_idle')
        _cpus = range(cpuidle_data.cpu_id.max() + 1)
    else:
        _cpus = listify(cpus)

    # Generate valid idle states
    idle_df = ia._dfg_trace_event('cpu_idle')
    available_idles = sorted(idle_df.state.unique())
    available_idles = available_idles[1:]
    
    residencies = pd.DataFrame()
    total_r = pd.DataFrame({'over_est' : 0, 'under_est' : 0, 'good_est' : 0, 'cpu' : -1}, index=available_idles)
    for cpu in _cpus:
        r = dfCPUIdleStateEstimationStatistics(ia, cpu, residency)
        residencies = pd.concat([residencies, r])
        total_r['over_est']  = total_r['over_est']  + r['over_est']
        total_r['under_est'] = total_r['under_est'] + r['under_est']
        total_r['good_est']  = total_r['good_est']  + r['good_est']
        
    residencies = pd.concat([residencies, total_r])
    return residencies

# Hikey620 Platform Configurations

In [5]:
# Setup target configuration
hikey620_conf = {
    "platform"     : 'linux',
    "board"        : 'hikey',
    "host"         : "192.168.1.20",
    "username"     : "root",
    "password"     : "root",
    "results_dir"  : "cpuidle_prev_opt",
    "modules"      : [ 'cpufreq', 'cpuidle' ],
    "ftrace"       : {
        "events" : [ "cpu_idle", "cpu_capacity", "cpu_frequency", "sched_switch", "sched_load_avg_cpu", "cpuidle_interval_stats", "cpuidle_state_idx",
                     "cpuidle_factor", "irq_handler_entry", "ipi_entry_rcuidle", "softirq_raise", "sched_migrate_task" ],
        "buffsize" : 50 * 1024,
    },
    "tools"        : [ 'trace-cmd', 'taskset', 'rt-app', 'sysbench' ],
    "rtapp-calib"  : {
        "0": 465, "1": 465, "2": 465, "3": 465, "4": 465, "5": 465, "6": 465, "7": 465,
    },    
}

In [6]:
hikey620_platform = {
    'kernel': {
        'major': 4,
        'sha1': '78e8b90',
        'version': '57 SMP PREEMPT Sat Jul 1 15:42:00 CST 2017',
        'version_number': 4,
        'parts': [4, 4, 74],
        'rc': None,
        'release': '4.4.74-g78e8b90',
        'minor': 74
    },
    
    'freqs': {
        'big':    [ 280000, 432000, 729000, 960000, 1200000 ],
        'little': [ 280000, 432000, 729000, 960000, 1200000 ]
    },
    
    'nrg_model': {
        'big': {
            'cluster': {
                'nrg_max': 112
            },
            'cpu': {
                'cap_max': 1024,
                'nrg_max': 607
            }
        },
        'little': {
            'cluster': {
                'nrg_max': 112
            },
            'cpu': {
                'cap_max': 1024,
                'nrg_max': 607
            }
        }
    },
    
    'clusters': {
        'big': [4, 5, 6, 7],
        'little': [0, 1, 2, 3]
    },
    'cpus_count': 8,
    'topology': [[0, 1, 2, 3], [4, 5, 6, 7]]
}

In [7]:
# Support to access the remote target
from env import TestEnv

te = TestEnv(hikey620_conf)
target = te.target

target.cpufreq.set_all_governors('userspace')
target.cpufreq.set_frequency(0, 1200000)

rfreq = target.cpufreq.get_frequency(0)
print "cpu frequency:", rfreq

2018-07-26 20:23:26,436 INFO    : TestEnv      : Using base path: /home/leoy/Work2/Develop/tools/lisa
2018-07-26 20:23:26,441 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-07-26 20:23:26,443 INFO    : TestEnv      : Devlib modules to load: ['cpuidle', 'cpufreq']
2018-07-26 20:23:26,444 INFO    : TestEnv      : Connecting linux target:
2018-07-26 20:23:26,448 INFO    : TestEnv      :   username : root
2018-07-26 20:23:26,449 INFO    : TestEnv      :       host : 192.168.1.20
2018-07-26 20:23:26,450 INFO    : TestEnv      :   password : root
2018-07-26 20:23:26,452 INFO    : TestEnv      : Connection settings:
2018-07-26 20:23:26,453 INFO    : TestEnv      :    {'username': 'root', 'host': '192.168.1.20', 'password': 'root'}
2018-07-26 20:23:26,462 DEBUG   : ssh          : Logging in root@192.168.1.20
2018-07-26 20:23:27,774 DEBUG   : ssh          : (id); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:23:27,949 DEBUG   : ssh          : (mkdir -p /root/

2018-07-26 20:23:36,973 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:23:37,136 DEBUG   : ssh          : (echo 1200000 > '/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed'); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:23:37,297 DEBUG   : ssh          : (cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq'); __devlib_ec=$?; echo; echo $__devlib_ec


cpu frequency: 1200000


# Workload execution utility

In [8]:
def execute(te, wload, res_dir):
    
    logging.info('# Create results folder for this execution')
    !mkdir {res_dir}
    
    logging.info('# Setup FTrace')
    te.ftrace.start()

    #logging.info('## Start energy sampling')
    #te.emeter.reset()

    logging.info('### Start RTApp execution')
    wload.run(out_dir=res_dir)

    #logging.info('## Read energy consumption: %s/energy.json', res_dir)
    #nrg_report = te.emeter.report(out_dir=res_dir)

    logging.info('# Stop FTrace')
    te.ftrace.stop()

    trace_file = os.path.join(res_dir, 'trace.dat')
    logging.info('# Save FTrace: %s', trace_file)
    te.ftrace.get_trace(trace_file)

    logging.info('# Save platform description: %s/platform.json', res_dir)
    plt, plt_file = te.platform_dump(res_dir)
    
    logging.info('# Report collected data:')
    logging.info('   %s', res_dir)
    !ls -la {res_dir}
    
    return plt, plt_file, trace_file

# Single task RTApp workload (period: 10ms, dc: 1%)

In [9]:
rtapp_name = 'example_10ms_1pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p1': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 1,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:23:37,405 INFO    : Workload     : Setup new workload example_10ms_1pct
2018-07-26 20:23:37,507 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:23:37,570 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:23:37,572 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:23:37,574 INFO    : Workload     : ------------------------
2018-07-26 20:23:37,576 INFO    : Workload     : task [task_p1], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:23:37,578 INFO    : Workload     :  | loops count: 1
2018-07-26 20:23:37,581 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:23:37,582 INFO    : Workload     : |  period    10000 [us], duty_cycle   1 %
2018-07-26 20:23:37,584 INFO    : Workload     : |  run_time    100 [us], sleep_time   9900 [us]
2018-07-26 20:23:37,586 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_1pct_0

total 7452
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:23 .
drwxrwxr-x 3 leoy leoy    4096 Jul 26 20:23 ..
-rw-r--r-- 1 leoy leoy     586 Jul 26 20:23 example_10ms_1pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:23 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:23 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:23 rt-app-task_p10-0.log
-rw-r--r-- 1 leoy leoy  124160 Jul 26 20:23 rt-app-task_p1-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:23 rt-app-task_p15-0.log
-rw-rw-r-- 1 leoy leoy 6983680 Jul 26 20:23 trace.dat


,cpu,good_est,over_est,under_est
0,0,93,0,8
1,0,5,23,48
2,0,91,65,0
0,1,42,0,17
1,1,15,20,71
2,1,562,34,0
0,2,33,0,10
1,2,15,33,64
2,2,125,19,0
0,3,26,0,7


# Single task RTApp workload (period: 5ms, dc: 1%)

In [10]:
rtapp_name = 'example_5ms_1pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p3': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 1,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:23:58,982 INFO    : Workload     : Setup new workload example_5ms_1pct
2018-07-26 20:23:59,085 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:23:59,147 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:23:59,149 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:23:59,151 INFO    : Workload     : ------------------------
2018-07-26 20:23:59,153 INFO    : Workload     : task [task_p3], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:23:59,157 INFO    : Workload     :  | loops count: 1
2018-07-26 20:23:59,161 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:23:59,162 INFO    : Workload     : |  period     5000 [us], duty_cycle   1 %
2018-07-26 20:23:59,164 INFO    : Workload     : |  run_time     50 [us], sleep_time   4950 [us]
2018-07-26 20:23:59,166 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_1pct_00.

total 8288
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:24 .
drwxrwxr-x 4 leoy leoy    4096 Jul 26 20:23 ..
-rw-r--r-- 1 leoy leoy     584 Jul 26 20:24 example_5ms_1pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:24 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:24 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:24 rt-app-task_p30-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:24 rt-app-task_p3-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:24 rt-app-task_p35-0.log
-rw-rw-r-- 1 leoy leoy 7716864 Jul 26 20:24 trace.dat


,cpu,good_est,over_est,under_est
0,0,198,0,15
1,0,12,40,48
2,0,73,207,0
0,1,32,0,15
1,1,13,26,77
2,1,56,41,0
0,2,35,0,11
1,2,10,14,47
2,2,85,9,0
0,3,894,0,8


# Single task RTApp workload (period: 10ms, dc: 3%)

In [11]:
rtapp_name = 'example_10ms_3pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p3': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 3,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:24:20,918 INFO    : Workload     : Setup new workload example_10ms_3pct
2018-07-26 20:24:21,020 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:24:21,084 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:24:21,086 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:24:21,089 INFO    : Workload     : ------------------------
2018-07-26 20:24:21,091 INFO    : Workload     : task [task_p3], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:24:21,095 INFO    : Workload     :  | loops count: 1
2018-07-26 20:24:21,097 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:24:21,098 INFO    : Workload     : |  period    10000 [us], duty_cycle   3 %
2018-07-26 20:24:21,100 INFO    : Workload     : |  run_time    300 [us], sleep_time   9700 [us]
2018-07-26 20:24:21,102 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_3pct_0

total 7360
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:24 .
drwxrwxr-x 5 leoy leoy    4096 Jul 26 20:24 ..
-rw-r--r-- 1 leoy leoy     586 Jul 26 20:24 example_10ms_3pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:24 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:24 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:24 rt-app-task_p30-0.log
-rw-r--r-- 1 leoy leoy  124160 Jul 26 20:24 rt-app-task_p3-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:24 rt-app-task_p35-0.log
-rw-rw-r-- 1 leoy leoy 6889472 Jul 26 20:24 trace.dat


,cpu,good_est,over_est,under_est
0,0,67,0,13
1,0,9,54,79
2,0,93,61,0
0,1,36,0,20
1,1,26,35,73
2,1,53,35,0
0,2,55,0,5
1,2,15,13,91
2,2,1065,20,0
0,3,24,0,10


# Single task RTApp workload (period: 5ms, dc: 3%)

In [12]:
rtapp_name = 'example_5ms_3pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p3': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 3,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:24:42,506 INFO    : Workload     : Setup new workload example_5ms_3pct
2018-07-26 20:24:42,608 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:24:42,671 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:24:42,672 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:24:42,674 INFO    : Workload     : ------------------------
2018-07-26 20:24:42,676 INFO    : Workload     : task [task_p3], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:24:42,678 INFO    : Workload     :  | loops count: 1
2018-07-26 20:24:42,679 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:24:42,681 INFO    : Workload     : |  period     5000 [us], duty_cycle   3 %
2018-07-26 20:24:42,682 INFO    : Workload     : |  run_time    150 [us], sleep_time   4850 [us]
2018-07-26 20:24:42,685 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_3pct_00.

total 8288
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:25 .
drwxrwxr-x 6 leoy leoy    4096 Jul 26 20:24 ..
-rw-r--r-- 1 leoy leoy     585 Jul 26 20:24 example_5ms_3pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:24 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:25 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:24 rt-app-task_p30-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:24 rt-app-task_p3-0.log
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:24 rt-app-task_p35-0.log
-rw-rw-r-- 1 leoy leoy 7716864 Jul 26 20:25 trace.dat


,cpu,good_est,over_est,under_est
0,0,258,0,16
1,0,11,54,68
2,0,95,151,0
0,1,1037,0,10
1,1,963,29,583
2,1,1532,4,0
0,2,40,0,9
1,2,15,14,49
2,2,113,22,0
0,3,32,0,11


# Single task RTApp workload (period: 10ms, dc: 5%)

In [13]:
rtapp_name = 'example_10ms_5pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p5': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 5,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:25:04,552 INFO    : Workload     : Setup new workload example_10ms_5pct
2018-07-26 20:25:04,654 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:25:04,718 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:25:04,720 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:25:04,722 INFO    : Workload     : ------------------------
2018-07-26 20:25:04,724 INFO    : Workload     : task [task_p5], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:25:04,727 INFO    : Workload     :  | loops count: 1
2018-07-26 20:25:04,730 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:25:04,732 INFO    : Workload     : |  period    10000 [us], duty_cycle   5 %
2018-07-26 20:25:04,733 INFO    : Workload     : |  run_time    500 [us], sleep_time   9500 [us]
2018-07-26 20:25:04,735 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_5pct_0

total 6824
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:25 .
drwxrwxr-x 7 leoy leoy    4096 Jul 26 20:25 ..
-rw-r--r-- 1 leoy leoy     586 Jul 26 20:25 example_10ms_5pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:25 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:25 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 26 20:25 rt-app-task_p5-0.log
-rw-rw-r-- 1 leoy leoy 6840320 Jul 26 20:25 trace.dat


,cpu,good_est,over_est,under_est
0,0,15,0,5
1,0,11,12,62
2,0,85,168,0
0,1,5,0,2
1,1,10,4,58
2,1,168,9,0
0,2,33,0,7
1,2,20,10,59
2,2,612,20,0
0,3,15,0,11


# Single task RTApp workload (period: 5ms, dc: 5%)

In [14]:
rtapp_name = 'example_5ms_5pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p5': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 5,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:25:25,823 INFO    : Workload     : Setup new workload example_5ms_5pct
2018-07-26 20:25:25,925 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:25:25,989 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:25:25,990 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:25:25,992 INFO    : Workload     : ------------------------
2018-07-26 20:25:25,993 INFO    : Workload     : task [task_p5], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:25:25,995 INFO    : Workload     :  | loops count: 1
2018-07-26 20:25:25,996 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:25:25,998 INFO    : Workload     : |  period     5000 [us], duty_cycle   5 %
2018-07-26 20:25:25,999 INFO    : Workload     : |  run_time    250 [us], sleep_time   4750 [us]
2018-07-26 20:25:26,001 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_5pct_00.

total 7588
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:25 .
drwxrwxr-x 8 leoy leoy    4096 Jul 26 20:25 ..
-rw-r--r-- 1 leoy leoy     585 Jul 26 20:25 example_5ms_5pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:25 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:25 platform.json
-rw-r--r-- 1 leoy leoy  248160 Jul 26 20:25 rt-app-task_p5-0.log
-rw-rw-r-- 1 leoy leoy 7499776 Jul 26 20:25 trace.dat


,cpu,good_est,over_est,under_est
0,0,169,0,3
1,0,58,40,56
2,0,200,99,0
0,1,937,0,6
1,1,528,0,117
2,1,1843,13,0
0,2,28,0,6
1,2,15,13,68
2,2,89,15,0
0,3,15,0,11


# Single task RTApp workload (period: 10ms, dc: 10%)

In [15]:
rtapp_name = 'example_10ms_10pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p10': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 10, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:25:47,465 INFO    : Workload     : Setup new workload example_10ms_10pct
2018-07-26 20:25:47,568 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:25:47,633 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:25:47,637 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:25:47,640 INFO    : Workload     : ------------------------
2018-07-26 20:25:47,642 INFO    : Workload     : task [task_p10], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:25:47,644 INFO    : Workload     :  | loops count: 1
2018-07-26 20:25:47,646 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:25:47,647 INFO    : Workload     : |  period    10000 [us], duty_cycle  10 %
2018-07-26 20:25:47,649 INFO    : Workload     : |  run_time   1000 [us], sleep_time   9000 [us]
2018-07-26 20:25:47,652 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_10pc

total 6804
drwxrwxr-x 2 leoy leoy    4096 Jul 26 20:26 .
drwxrwxr-x 9 leoy leoy    4096 Jul 26 20:25 ..
-rw-r--r-- 1 leoy leoy     589 Jul 26 20:26 example_10ms_10pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 26 20:26 output.log
-rw-rw-r-- 1 leoy leoy    1330 Jul 26 20:26 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 26 20:26 rt-app-task_p10-0.log
-rw-rw-r-- 1 leoy leoy 6819840 Jul 26 20:26 trace.dat


,cpu,good_est,over_est,under_est
0,0,33,0,11
1,0,12,32,64
2,0,1066,88,0
0,1,12,0,5
1,1,17,7,63
2,1,55,10,0
0,2,16,0,4
1,2,9,8,72
2,2,86,12,0
0,3,5,0,3


# Single task RTApp workload (period: 5ms, dc: 10%)

In [16]:
rtapp_name = 'example_5ms_10pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p10': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 10, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:26:08,445 INFO    : Workload     : Setup new workload example_5ms_10pct
2018-07-26 20:26:08,548 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:26:08,612 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:26:08,613 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:26:08,617 INFO    : Workload     : ------------------------
2018-07-26 20:26:08,620 INFO    : Workload     : task [task_p10], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:26:08,624 INFO    : Workload     :  | loops count: 1
2018-07-26 20:26:08,625 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:26:08,627 INFO    : Workload     : |  period     5000 [us], duty_cycle  10 %
2018-07-26 20:26:08,635 INFO    : Workload     : |  run_time    500 [us], sleep_time   4500 [us]
2018-07-26 20:26:08,637 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_10pct_

total 7772
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:26 .
drwxrwxr-x 10 leoy leoy    4096 Jul 26 20:26 ..
-rw-r--r--  1 leoy leoy     587 Jul 26 20:26 example_5ms_10pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:26 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:26 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:26 rt-app-task_p10-0.log
-rw-rw-r--  1 leoy leoy 7688192 Jul 26 20:26 trace.dat


,cpu,good_est,over_est,under_est
0,0,263,0,11
1,0,11,18,48
2,0,86,132,0
0,1,36,0,8
1,1,10,24,61
2,1,46,19,0
0,2,37,0,11
1,2,7,23,66
2,2,101,15,0
0,3,13,0,1


# Single task RTApp workload (period: 10ms, dc: 15%)

In [17]:
rtapp_name = 'example_10ms_15pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p15': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 15, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:26:29,968 INFO    : Workload     : Setup new workload example_10ms_15pct
2018-07-26 20:26:30,070 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:26:30,133 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:26:30,134 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:26:30,136 INFO    : Workload     : ------------------------
2018-07-26 20:26:30,138 INFO    : Workload     : task [task_p15], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:26:30,140 INFO    : Workload     :  | loops count: 1
2018-07-26 20:26:30,143 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:26:30,145 INFO    : Workload     : |  period    10000 [us], duty_cycle  15 %
2018-07-26 20:26:30,147 INFO    : Workload     : |  run_time   1500 [us], sleep_time   8500 [us]
2018-07-26 20:26:30,148 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_15pc

total 6852
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:26 .
drwxrwxr-x 11 leoy leoy    4096 Jul 26 20:26 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:26 example_10ms_15pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:26 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:26 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:26 rt-app-task_p15-0.log
-rw-rw-r--  1 leoy leoy 6868992 Jul 26 20:26 trace.dat


,cpu,good_est,over_est,under_est
0,0,34,0,8
1,0,12,24,50
2,0,304,102,0
0,1,43,0,19
1,1,15,24,47
2,1,65,28,0
0,2,26,0,12
1,2,12,25,64
2,2,398,9,0
0,3,23,0,7


# Single task RTApp workload (period: 5ms, dc: 15%)

In [18]:
rtapp_name = 'example_5ms_15pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p15': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 15, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:26:50,676 INFO    : Workload     : Setup new workload example_5ms_15pct
2018-07-26 20:26:50,778 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:26:50,841 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:26:50,843 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:26:50,845 INFO    : Workload     : ------------------------
2018-07-26 20:26:50,847 INFO    : Workload     : task [task_p15], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:26:50,849 INFO    : Workload     :  | loops count: 1
2018-07-26 20:26:50,851 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:26:50,852 INFO    : Workload     : |  period     5000 [us], duty_cycle  15 %
2018-07-26 20:26:50,854 INFO    : Workload     : |  run_time    750 [us], sleep_time   4250 [us]
2018-07-26 20:26:50,856 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_15pct_

total 7672
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:27 .
drwxrwxr-x 12 leoy leoy    4096 Jul 26 20:26 ..
-rw-r--r--  1 leoy leoy     587 Jul 26 20:27 example_5ms_15pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:27 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:27 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:27 rt-app-task_p15-0.log
-rw-rw-r--  1 leoy leoy 7585792 Jul 26 20:27 trace.dat


,cpu,good_est,over_est,under_est
0,0,28,0,6
1,0,10,10,51
2,0,82,91,0
0,1,19,0,8
1,1,17,6,69
2,1,65,30,0
0,2,1018,0,9
1,2,1502,6,596
2,2,997,21,0
0,3,8,0,5


# Single task RTApp workload (period: 10ms, dc: 20%)

In [19]:
rtapp_name = 'example_10ms_20pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p20': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 20, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:27:12,429 INFO    : Workload     : Setup new workload example_10ms_20pct
2018-07-26 20:27:12,539 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:27:12,601 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:27:12,603 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:27:12,605 INFO    : Workload     : ------------------------
2018-07-26 20:27:12,606 INFO    : Workload     : task [task_p20], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:27:12,608 INFO    : Workload     :  | loops count: 1
2018-07-26 20:27:12,610 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:27:12,612 INFO    : Workload     : |  period    10000 [us], duty_cycle  20 %
2018-07-26 20:27:12,614 INFO    : Workload     : |  run_time   2000 [us], sleep_time   8000 [us]
2018-07-26 20:27:12,616 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_20pc

total 6860
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:27 .
drwxrwxr-x 13 leoy leoy    4096 Jul 26 20:27 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:27 example_10ms_20pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:27 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:27 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:27 rt-app-task_p20-0.log
-rw-rw-r--  1 leoy leoy 6877184 Jul 26 20:27 trace.dat


,cpu,good_est,over_est,under_est
0,0,35,0,7
1,0,13,24,80
2,0,299,76,0
0,1,16,0,8
1,1,9,9,46
2,1,1051,24,0
0,2,11,0,8
1,2,10,12,42
2,2,94,22,0
0,3,9,0,6


# Single task RTApp workload (period: 5ms, dc: 20%)

In [20]:
rtapp_name = 'example_5ms_20pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p20': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 20, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:27:33,342 INFO    : Workload     : Setup new workload example_5ms_20pct
2018-07-26 20:27:33,444 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:27:33,508 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:27:33,509 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:27:33,511 INFO    : Workload     : ------------------------
2018-07-26 20:27:33,512 INFO    : Workload     : task [task_p20], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:27:33,514 INFO    : Workload     :  | loops count: 1
2018-07-26 20:27:33,516 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:27:33,518 INFO    : Workload     : |  period     5000 [us], duty_cycle  20 %
2018-07-26 20:27:33,520 INFO    : Workload     : |  run_time   1000 [us], sleep_time   4000 [us]
2018-07-26 20:27:33,522 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_20pct_

total 7616
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:27 .
drwxrwxr-x 14 leoy leoy    4096 Jul 26 20:27 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:27 example_5ms_20pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:27 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:27 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:27 rt-app-task_p20-0.log
-rw-rw-r--  1 leoy leoy 7528448 Jul 26 20:27 trace.dat


,cpu,good_est,over_est,under_est
0,0,49,0,9
1,0,11,27,64
2,0,95,62,0
0,1,27,0,11
1,1,15,17,56
2,1,44,36,0
0,2,552,0,496
1,2,999,15,1564
2,2,43,12,0
0,3,31,0,6


# Single task RTApp workload (period: 10ms, dc: 25%)

In [21]:
rtapp_name = 'example_10ms_25pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p25': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 25, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:27:55,000 INFO    : Workload     : Setup new workload example_10ms_25pct
2018-07-26 20:27:55,102 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:27:55,166 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:27:55,169 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:27:55,171 INFO    : Workload     : ------------------------
2018-07-26 20:27:55,174 INFO    : Workload     : task [task_p25], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:27:55,177 INFO    : Workload     :  | loops count: 1
2018-07-26 20:27:55,179 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:27:55,180 INFO    : Workload     : |  period    10000 [us], duty_cycle  25 %
2018-07-26 20:27:55,182 INFO    : Workload     : |  run_time   2500 [us], sleep_time   7500 [us]
2018-07-26 20:27:55,183 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_25pc

total 6896
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:28 .
drwxrwxr-x 15 leoy leoy    4096 Jul 26 20:27 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:28 example_10ms_25pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:28 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:28 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:28 rt-app-task_p25-0.log
-rw-rw-r--  1 leoy leoy 6914048 Jul 26 20:28 trace.dat


,cpu,good_est,over_est,under_est
0,0,55,0,15
1,0,12,34,81
2,0,310,124,0
0,1,25,0,8
1,1,12,25,51
2,1,60,22,0
0,2,18,0,4
1,2,2,15,72
2,2,126,16,0
0,3,13,0,6


# Single task RTApp workload (period: 5ms, dc: 25%)

In [22]:
rtapp_name = 'example_5ms_25pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p25': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 25, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:28:16,108 INFO    : Workload     : Setup new workload example_5ms_25pct
2018-07-26 20:28:16,210 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:28:16,274 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:28:16,275 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:28:16,277 INFO    : Workload     : ------------------------
2018-07-26 20:28:16,278 INFO    : Workload     : task [task_p25], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:28:16,279 INFO    : Workload     :  | loops count: 1
2018-07-26 20:28:16,281 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:28:16,282 INFO    : Workload     : |  period     5000 [us], duty_cycle  25 %
2018-07-26 20:28:16,283 INFO    : Workload     : |  run_time   1250 [us], sleep_time   3750 [us]
2018-07-26 20:28:16,286 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_25pct_

total 7680
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:28 .
drwxrwxr-x 16 leoy leoy    4096 Jul 26 20:28 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:28 example_5ms_25pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:28 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:28 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:28 rt-app-task_p25-0.log
-rw-rw-r--  1 leoy leoy 7593984 Jul 26 20:28 trace.dat


,cpu,good_est,over_est,under_est
0,0,93,0,5
1,0,8,34,52
2,0,114,94,0
0,1,15,0,2
1,1,5,10,72
2,1,65,6,0
0,2,1006,0,501
1,2,509,4,1578
2,2,79,13,0
0,3,8,0,5


# Single task RTApp workload (period: 10ms, dc: 30%)

In [23]:
rtapp_name = 'example_10ms_30pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p30': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 30, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:28:37,946 INFO    : Workload     : Setup new workload example_10ms_30pct
2018-07-26 20:28:38,048 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:28:38,112 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:28:38,115 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:28:38,118 INFO    : Workload     : ------------------------
2018-07-26 20:28:38,120 INFO    : Workload     : task [task_p30], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:28:38,121 INFO    : Workload     :  | loops count: 1
2018-07-26 20:28:38,123 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:28:38,124 INFO    : Workload     : |  period    10000 [us], duty_cycle  30 %
2018-07-26 20:28:38,126 INFO    : Workload     : |  run_time   3000 [us], sleep_time   7000 [us]
2018-07-26 20:28:38,128 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_30pc

total 6904
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:28 .
drwxrwxr-x 17 leoy leoy    4096 Jul 26 20:28 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:28 example_10ms_30pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:28 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:28 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:28 rt-app-task_p30-0.log
-rw-rw-r--  1 leoy leoy 6922240 Jul 26 20:28 trace.dat


,cpu,good_est,over_est,under_est
0,0,22,0,10
1,0,13,8,41
2,0,312,199,0
0,1,8,0,5
1,1,8,2,58
2,1,75,22,0
0,2,25,0,3
1,2,32,8,65
2,2,1047,20,0
0,3,9,0,4


# Single task RTApp workload (period: 5ms, dc: 30%)

In [24]:
rtapp_name = 'example_5ms_30pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p30': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 30, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:28:59,007 INFO    : Workload     : Setup new workload example_5ms_30pct
2018-07-26 20:28:59,109 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:28:59,171 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:28:59,173 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:28:59,175 INFO    : Workload     : ------------------------
2018-07-26 20:28:59,177 INFO    : Workload     : task [task_p30], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:28:59,178 INFO    : Workload     :  | loops count: 1
2018-07-26 20:28:59,181 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:28:59,183 INFO    : Workload     : |  period     5000 [us], duty_cycle  30 %
2018-07-26 20:28:59,184 INFO    : Workload     : |  run_time   1500 [us], sleep_time   3500 [us]
2018-07-26 20:28:59,186 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_30pct_

total 7604
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:29 .
drwxrwxr-x 18 leoy leoy    4096 Jul 26 20:28 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:29 example_5ms_30pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:29 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:29 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:29 rt-app-task_p30-0.log
-rw-rw-r--  1 leoy leoy 7516160 Jul 26 20:29 trace.dat


,cpu,good_est,over_est,under_est
0,0,99,0,7
1,0,14,33,45
2,0,83,87,0
0,1,30,0,12
1,1,13,26,71
2,1,65,24,0
0,2,33,0,15
1,2,15,26,60
2,2,75,23,0
0,3,1027,0,508


# Single task RTApp workload (period: 10ms, dc: 35%)

In [25]:
rtapp_name = 'example_10ms_35pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p35': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 35, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:29:20,389 INFO    : Workload     : Setup new workload example_10ms_35pct
2018-07-26 20:29:20,498 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:29:20,561 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:29:20,562 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:29:20,564 INFO    : Workload     : ------------------------
2018-07-26 20:29:20,566 INFO    : Workload     : task [task_p35], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:29:20,568 INFO    : Workload     :  | loops count: 1
2018-07-26 20:29:20,570 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:29:20,572 INFO    : Workload     : |  period    10000 [us], duty_cycle  35 %
2018-07-26 20:29:20,573 INFO    : Workload     : |  run_time   3500 [us], sleep_time   6500 [us]
2018-07-26 20:29:20,575 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_35pc

total 6852
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:29 .
drwxrwxr-x 19 leoy leoy    4096 Jul 26 20:29 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:29 example_10ms_35pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:29 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:29 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:29 rt-app-task_p35-0.log
-rw-rw-r--  1 leoy leoy 6868992 Jul 26 20:29 trace.dat


,cpu,good_est,over_est,under_est
0,0,20,0,3
1,0,19,13,53
2,0,303,129,0
0,1,17,0,7
1,1,16,14,69
2,1,57,18,0
0,2,25,0,6
1,2,26,8,47
2,2,1055,19,0
0,3,8,0,5


# Single task RTApp workload (period: 5ms, dc: 35%)

In [26]:
rtapp_name = 'example_5ms_35pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p35': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 35, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:29:41,245 INFO    : Workload     : Setup new workload example_5ms_35pct
2018-07-26 20:29:41,348 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:29:41,412 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:29:41,414 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:29:41,416 INFO    : Workload     : ------------------------
2018-07-26 20:29:41,418 INFO    : Workload     : task [task_p35], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:29:41,419 INFO    : Workload     :  | loops count: 1
2018-07-26 20:29:41,421 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:29:41,422 INFO    : Workload     : |  period     5000 [us], duty_cycle  35 %
2018-07-26 20:29:41,424 INFO    : Workload     : |  run_time   1750 [us], sleep_time   3250 [us]
2018-07-26 20:29:41,426 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_35pct_

total 7656
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:30 .
drwxrwxr-x 20 leoy leoy    4096 Jul 26 20:29 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:29 example_5ms_35pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:29 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:30 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:29 rt-app-task_p35-0.log
-rw-rw-r--  1 leoy leoy 7569408 Jul 26 20:30 trace.dat


,cpu,good_est,over_est,under_est
0,0,71,0,13
1,0,7,64,67
2,0,96,102,0
0,1,1025,0,507
1,1,1013,24,1071
2,1,39,11,0
0,2,10,0,10
1,2,12,10,74
2,2,95,25,0
0,3,18,0,7


# Single task RTApp workload (period: 10ms, dc: 40%)

In [27]:
rtapp_name = 'example_10ms_40pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p40': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 40, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:30:03,137 INFO    : Workload     : Setup new workload example_10ms_40pct
2018-07-26 20:30:03,240 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:30:03,302 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:30:03,304 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:30:03,306 INFO    : Workload     : ------------------------
2018-07-26 20:30:03,307 INFO    : Workload     : task [task_p40], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:30:03,309 INFO    : Workload     :  | loops count: 1
2018-07-26 20:30:03,310 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-26 20:30:03,312 INFO    : Workload     : |  period    10000 [us], duty_cycle  40 %
2018-07-26 20:30:03,313 INFO    : Workload     : |  run_time   4000 [us], sleep_time   6000 [us]
2018-07-26 20:30:03,315 DEBUG   : ssh          : /usr/bin/scp -r   example_10ms_40pc

total 6916
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:30 .
drwxrwxr-x 21 leoy leoy    4096 Jul 26 20:30 ..
-rw-r--r--  1 leoy leoy     589 Jul 26 20:30 example_10ms_40pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:30 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:30 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 26 20:30 rt-app-task_p40-0.log
-rw-rw-r--  1 leoy leoy 6934528 Jul 26 20:30 trace.dat


,cpu,good_est,over_est,under_est
0,0,96,0,2
1,0,29,6,48
2,0,770,142,0
0,1,75,0,15
1,1,13,9,43
2,1,196,11,0
0,2,24,0,5
1,2,14,14,76
2,2,132,36,0
0,3,13,0,3


# Single task RTApp workload (period: 5ms, dc: 40%)

In [28]:
rtapp_name = 'example_5ms_40pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p40': Periodic (
            period_ms      = 5, # period [ms]
            duty_cycle_pct = 40, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-26 20:30:23,895 INFO    : Workload     : Setup new workload example_5ms_40pct
2018-07-26 20:30:23,997 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-26 20:30:24,060 INFO    : Workload     : Workload duration defined by longest task
2018-07-26 20:30:24,062 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-26 20:30:24,064 INFO    : Workload     : ------------------------
2018-07-26 20:30:24,066 INFO    : Workload     : task [task_p40], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-26 20:30:24,068 INFO    : Workload     :  | loops count: 1
2018-07-26 20:30:24,070 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (2000 loops)
2018-07-26 20:30:24,071 INFO    : Workload     : |  period     5000 [us], duty_cycle  40 %
2018-07-26 20:30:24,073 INFO    : Workload     : |  run_time   2000 [us], sleep_time   3000 [us]
2018-07-26 20:30:24,074 DEBUG   : ssh          : /usr/bin/scp -r   example_5ms_40pct_

total 7692
drwxrwxr-x  2 leoy leoy    4096 Jul 26 20:30 .
drwxrwxr-x 22 leoy leoy    4096 Jul 26 20:30 ..
-rw-r--r--  1 leoy leoy     588 Jul 26 20:30 example_5ms_40pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 26 20:30 output.log
-rw-rw-r--  1 leoy leoy    1330 Jul 26 20:30 platform.json
-rw-r--r--  1 leoy leoy  248160 Jul 26 20:30 rt-app-task_p40-0.log
-rw-rw-r--  1 leoy leoy 7606272 Jul 26 20:30 trace.dat


,cpu,good_est,over_est,under_est
0,0,33,0,5
1,0,13,19,67
2,0,105,215,0
0,1,155,0,13
1,1,16,9,70
2,1,66,31,0
0,2,7,0,6
1,2,10,7,49
2,2,84,13,0
0,3,1017,0,510
